In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from datasets import load_dataset
from uk_stemmer import UkStemmer
import re
from typing import List
from nltk.tokenize import word_tokenize
from string import punctuation

In [9]:
ds = load_dataset('Yehor/ukrainian-news-headlines', split='train')

Generating train split: 5242391 examples [00:03, 1618234.82 examples/s]


In [10]:
ds

Dataset({
    features: ['headline', 'length'],
    num_rows: 5242391
})

In [13]:
df = pd.DataFrame(ds)

In [27]:
df.sample(frac=1).head(2000000)[['headline']].dropna().to_csv('./data/ukr-cut.csv', index=False)

In [31]:
df = pd.read_csv('./data/ukr-cut.csv')

df

,headline
0,Збитки на понад 9 мільйонів євро: у Дніпрі пій...
1,Друзі при Балтії
2,Обласна влада привітала жінок концертом Зібров...
3,Кличко доручив створити робочу групу і навести...
4,Вчені прирівняли переїдання до наркоманії
...,...
1769208,Відео
1769209,Міністерство культури зарубало конкурс Хоткеви...
1769210,Янукович велел ГПУ разобраться со смертью черн...
1769211,"Пенсіонерам, які працюють, збережуть усі соціа..."


In [40]:
stemmer = UkStemmer()

incorrect_stopwords = ['рос']

with open('./data/stopwords_ua.txt') as f_stopwords:
    stopwords = [x.replace('\n', '') for x in f_stopwords.readlines()]
    stopwords = [sw for sw in stopwords if sw not in incorrect_stopwords]

num_re = re.compile(r'\d+')
words_re = re.compile(r'\w+')

def prepare_uk(text: str) -> str:
    tokens = word_tokenize(text.lower())
    tokens = [stemmer.stem_word(t) for t in tokens]
    tokens = [t for t in tokens if t not in punctuation]
    tokens = [t for t in tokens if t not in stopwords]
    tokens = [t for t in tokens if num_re.match(t) is None]
    tokens = [t for t in tokens if words_re.match(t) is not None]

    return ' '.join(tokens)

In [47]:
tqdm.pandas()

In [48]:
df['headline_prcessed'] = df['headline'].progress_apply(prepare_uk)

df

100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 1769213/1769213 [10:20<00:00, 2848.98it/s]


,headline,headline_prcessed
0,Збитки на понад 9 мільйонів євро: у Дніпрі пій...,збитк мільйон євр дніпр пійм учасник міжнародн...
1,Друзі при Балтії,друз балті
2,Обласна влада привітала жінок концертом Зібров...,обласн влад привітал жінок концерт зібр братів...
3,Кличко доручив створити робочу групу і навести...,кличк доруч створит робоч груп навест лад буді...
4,Вчені прирівняли переїдання до наркоманії,вчен прирівнял переїданн наркомані
...,...,...
1769208,Відео,віде
1769209,Міністерство культури зарубало конкурс Хоткеви...,міністерств культур зарубал конкурс хоткевич харк
1769210,Янукович велел ГПУ разобраться со смертью черн...,янукович велел гпу разобр со смерт чернобыльц
1769211,"Пенсіонерам, які працюють, збережуть усі соціа...",пенсіонер працюют збережут соціальн випл мінсо...


In [52]:
df.to_csv('./data/uk-cut-processed.csv', index=False)